# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

In [8]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Inputs & Data

Explanation of each input and where it comes from.

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention/results/2024_01_30_rce2_spectral_preprocessing/proc/rce_pilot_2_04_spectral_and_sleap.pkl")

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [10]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 930], [930, 948...","[[32974, 32976], [33201, 33207], [33208, 33211...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[48.01956210909252, 37.264554576109816]","[33.1534452699471, 35.61101869926864]","[62.60288905131385, 35.71974322144712]",None,"[[[27.67316905433137, 31.06298795228111], [27....",NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,"[21.36, 21.36, 21.36, 21.36, 21.36, 21.36, 21....",NaN
2,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",None,"[[[34.36404036584844, 35.615553316175784], [34...",NaN,"[55.4, 52.03, 48.72, 45.53, 42.44, 39.44, 36.5...",NaN,"[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",NaN
3,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.1,"[[[34.36404036584844, 35.615553316175784], [34...","[[[46.68890098287189, 11.495085446056166], [43...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1..."
4,2,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,1.1,...,"[45.31189572048577, 36.38317068198775]","[30.490085836306232, 34.91627159792731]","[60.10744182471764, 34.262899786598894]",None,"[[[46.71964110607613, 36.493379374373234], [44...",NaN,"[5.8, 5.207, 4.64, 4.1, 3.596, 3.127, 2.7, 2.3...",NaN,"[4.633, 4.09, 4.36, 4.656, 4.65, 4.383, 4.35, ...",NaN


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [12]:
FULL_LFP_TRACES_PKL = "{}_05_lfp_spectral_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [13]:
ALL_PHY_DIR

['/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omi

In [14]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [15]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,3,71,22,243,8.729426,1,0,20.0,9.474292,mua,33069,0,5
1,5,4,0,14,9.769127,2,0,40.0,1.879731,mua,6561,0,7
2,6,2,0,11,13.532221,2,0,40.0,11.421351,good,39865,0,8
3,7,58,30,99,9.302568,2,0,40.0,20.851808,good,72781,0,9
4,8,29,12,94,7.241092,3,0,60.0,4.613521,mua,16103,0,10


- Combining all the unit info dataframes and adding the recording name

In [16]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [17]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,0,12,9,45,10.498872,0,0.0,0.0,4.379580,mua,14970,0,1.0
1,20230620_114347_standard_comp_to_omission_D4_s...,11,0,0,0,10.311028,5,0.0,100.0,0.230828,mua,789,0,12.0
2,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
3,20230620_114347_standard_comp_to_omission_D4_s...,21,148,57,355,9.774931,11,0.0,220.0,10.004284,mua,34196,0,23.0
4,20230620_114347_standard_comp_to_omission_D4_s...,22,105,26,351,8.435524,12,0.0,240.0,13.887395,mua,47469,0,24.0


- Filtering for the good units

In [18]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [19]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
1,20230620_114347_standard_comp_to_omission_D4_s...,28,2,1,7,8.708598,14,0.0,280.0,1.436163,good,4909,0,30.0
2,20230620_114347_standard_comp_to_omission_D4_s...,41,2,1,3,10.409801,19,0.0,380.0,1.583319,good,5412,0,43.0
3,20230620_114347_standard_comp_to_omission_D4_s...,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0
4,20230620_114347_standard_comp_to_omission_D4_s...,43,1,0,4,7.047093,26,0.0,520.0,1.489701,good,5092,0,45.0


In [20]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [21]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_clusters.npy'


In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 42,  42, 207, ...,  22, 229,  68], dtype=int32)

In [23]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(903326,)

- The times that all the spikes happened

In [24]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_times.npy'


In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[     120],
       [     262],
       [     439],
       ...,
       [68362487],
       [68362524],
       [68362610]])

In [26]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(903326, 1)

### Combining everything into a dataframe

In [27]:
recording_to_spike_df = {}

for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [28]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,2,0,0,0,24.692951,1,0.0,20.0,1.070175,good,3658,0,3.0
1,3,235,106,472,10.347874,1,0.0,20.0,14.493880,mua,49542,0,4.0
2,4,64,25,149,11.953332,4,0.0,80.0,27.163371,mua,92848,0,5.0
3,10,69,29,194,10.297276,18,0.0,360.0,6.455279,mua,22065,0,12.0
4,11,619,261,1295,9.638424,19,0.0,380.0,18.195901,mua,62196,0,13.0


In [29]:
spike_times

array([[     160],
       [     483],
       [     503],
       ...,
       [68361973],
       [68362204],
       [68362381]])

In [30]:
spike_clusters

array([ 4,  4, 11, ...,  4,  4, 87], dtype=int32)

- Combining the spike time df for all recordings

In [31]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [32]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [33]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,42,120,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
1,42,262,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,142.0,0.00710
2,207,439,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
3,207,632,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,193.0,0.00965
4,262,723,262,0,0,0,20.012720,19,0.0,380.0,2.037368,good,6964,0,39.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN


In [34]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
2017369,64,68356117,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,392.0,0.0196
2017370,89,68356258,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,3360.0,0.1680
2017371,64,68357205,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1088.0,0.0544
2017372,89,68357578,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,1320.0,0.0660
2017373,64,68358945,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1740.0,0.0870


## Grouping all the neurons by recording

In [35]:
# Grouping all spike times by neuron and recording
grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: np.array(x)).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)

In [36]:
grouped_df.head()

,spike_clusters,recording_name,spike_times
0,3,20230617_115521_standard_comp_to_omission_D1_s...,"[673223, 2367433, 2569212, 3969647, 3973243, 3..."
1,4,20230617_115521_standard_comp_to_omission_D1_s...,"[1071, 3665, 4592, 8167, 8358, 8807, 9043, 990..."
2,5,20230617_115521_standard_comp_to_omission_D1_s...,"[90780, 93969, 96069, 145651, 149204, 150749, ..."
3,6,20230617_115521_standard_comp_to_omission_D1_s...,"[102, 1962, 3893, 15127, 31864, 36704, 38461, ..."
4,17,20230617_115521_standard_comp_to_omission_D1_s...,"[151072, 201333, 206934, 213398, 220860, 23372..."


In [37]:
max_number_of_spikes = all_spike_time_df["n_spikes"].max()

In [38]:
max_number_of_spikes

179245

In [39]:
grouped_df["spike_times"] = grouped_df["spike_times"].apply(lambda x: np.concatenate([x, np.full([max_number_of_spikes - x.shape[0]], np.nan)]))

In [40]:
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: np.array(list(x))}).reset_index()

In [41]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,None,"[[[40.039741738777444, 9.47029992556593], [42....",NaN,"[14.84, 13.97, 13.13, 12.32, 11.54, 10.78, 10....",NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,1.2,"[[[57.86163468579171, 31.554562356382696], [59...","[[[40.039741738777444, 9.47029992556593], [42....","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3..."
2,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,1.1,"[[[40.039741738777444, 9.47029992556593], [42....","[[[57.86163468579171, 31.554562356382696], [59...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550..."
3,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,None,"[[[48.82535585060548, 37.83187483510619], [47....",NaN,"[5.184, 4.777, 4.383, 4.0, 3.629, 3.271, 2.926...",NaN,"[6.223, 6.227, 6.234, 6.24, 6.203, 6.2, 6.203,...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550..."
4,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,1.1,...,1.4,"[[[46.838967596646405, 35.8797798363605], [44....","[[[31.37122566669405, 31.1085619584881], [33.5...","[1.143, 1.143, 1.142, 1.139, 1.135, 1.129, 1.1...","[0.6333, 0.5635, 0.4963, 0.4321, 0.3713, 0.313...","[5.72

## Calculating the firing rates

In [43]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'agent', 'subject_locations', 'agent_locations',
       'subject_thorax_velocity', 'agent_thorax_velocity',
       'subject_thorax_to_reward_port', 'agent_thorax_to_reward_port',
       'recording_name', 'spike_clusters', 'spike_times'],
      dtype='object', length=156)

In [44]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,2,0,0,0,24.692951,1,0.0,20.0,1.070175,good,3658,0,3.0
1,3,235,106,472,10.347874,1,0.0,20.0,14.493880,mua,49542,0,4.0
2,4,64,25,149,11.953332,4,0.0,80.0,27.163371,mua,92848,0,5.0
3,10,69,29,194,10.297276,18,0.0,360.0,6.455279,mua,22065,0,12.0
4,11,619,261,1295,9.638424,19,0.0,380.0,18.195901,mua,62196,0,13.0


In [45]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,None,"[[[40.039741738777444, 9.47029992556593], [42....",NaN,"[14.84, 13.97, 13.13, 12.32, 11.54, 10.78, 10....",NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,1.2,"[[[57.86163468579171, 31.554562356382696], [59...","[[[40.039741738777444, 9.47029992556593], [42....","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3..."
2,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,1.1,"[[[40.039741738777444, 9.47029992556593], [42....","[[[57.86163468579171, 31.554562356382696], [59...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550..."
3,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,None,"[[[48.82535585060548, 37.83187483510619], [47....",NaN,"[5.184, 4.777, 4.383, 4.0, 3.629, 3.271, 2.926...",NaN,"[6.223, 6.227, 6.234, 6.24, 6.203, 6.2, 6.203,...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550..."
4,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,1.1,...,1.4,"[[[46.838967596646405, 35.8797798363605], [44....","[[[31.37122566669405, 31.1085619584881], [33.5...","[1.143, 1.143, 1.142, 1.139, 1.135, 1.129, 1.1...","[0.6333, 0.5635, 0.4963, 0.4321, 0.3713, 0.313...","[5.72

In [46]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times[~np.isnan(times)]), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)


In [47]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0][~np.isnan(x["spike_times"][0])], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [48]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [49]:
LFP_SPECTRAL_DF.iloc[:5,:10]

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1
2,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2
3,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2
4,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,1.1


In [50]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace
0,"[1.1, 1.2]",3478533,83177118,"[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.07367, 0.1105, 0.19, 0.2365, 0.0717, -0.160...","[0.1039, -0.011925, 0.02043, 0.1362, 0.011925,..."
1,"[1.1, 1.2]",3478533,83177118,"[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.07367, 0.1105, 0.19, 0.2365, 0.0717, -0.160...","[0.1039, -0.011925, 0.02043, 0.1362, 0.011925,..."
2,"[1.1, 1.2]",3478533,83177118,"[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.9487, 1.179, 1.587, 1.677, 1.536, 1.568, 1....","[0.8105, 0.909, 0.8447, 0.6675, 0.914, 1.372, ..."
3,"[1.1, 1.2]",3478533,83177118,"[40669726, 40671112, 40671112, 40672498, 40673...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.9487, 1.179, 1.587, 1.677, 1.536, 1.568, 1....","[0.8105, 0.909, 0.8447, 0.6675, 0.914, 1.372, ..."
4,"[1.1, 1.4]",835680,66439621,"[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[[33057437, 33069437], [33070037, 33075039], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.3154, -0.2812, -0.2698, -0.3381, -0.2261, ...","[0.003227, -0.06616, -0.0742, 0.0565, 0.1743, ..."


In [51]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,"[0.004, -0.1181, -0.05203, 0.1702, 0.1521, 0.0...","[0.2854, 0.8105, 1.242, 1.026, 0.6797, 0.294, ...","[-0.2625, -0.1997, -0.01572, 0.1509, 0.02202, ...","[[0.0237, 0.01284, 0.02682, 0.04108, 0.05316, ...","[[0.002655, 0.01074, 0.01842, 0.02864, 0.0338,...","[[0.00533, 0.01519, 0.0261, 0.03506, 0.0361, 0...","[[0.03333, 0.01772, 0.02557, 0.019, 0.02469, 0...","[[0.00755, 0.005825, 0.0322, 0.04767, 0.0523, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.004, -0.1181, -0.05203, 0.1702, 0.1521, 0.0...","[0.2854, 0.8105, 1.242, 1.026, 0.6797, 0.294, ...","[-0.2625, -0.1997, -0.01572, 0.1509, 0.02202, ...","[[0.0237, 0.01284, 0.02682, 0.04108, 0.05316, ...","[[0.002655, 0.01074, 0.01842, 0.02864, 0.0338,...","[[0.00533, 0.01519, 0.0261, 0.03506, 0.0361, 0...","[[0.03333, 0.01772, 0.02557, 0.019, 0.02469, 0...","[[0.00755, 0.005825, 0.0322, 0.04767, 0.0523, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[1.025, 1.567, 1.864, 1.6455, 1.957, 2.387, 2....","[0.577, 1.227, 1.856, 1.917, 2.18, 2.404, 2.46...","[-0.5967, -0.5806, -0.5005, -0.3042, 0.0, 0.18...","[[0.03217, 0.03186, 0.07117, 0.0814, 0.08716, ...","[[0.08264, 0.06064, 0.0753, 0.0392, 0.03656, 0...","[[0.01704, 0.0323, 0.04984, 0.05484, 0.05585, ...","[[0.004166, 0.0266, 0.0355, 0.0466, 0.05707, 0...","[[0.001713, 0.01749, 0.01785, 0.01508, 0.01006...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[1.025, 1.567, 1.864, 1.6455, 1.957, 2.387, 2....","[0.577, 1.227, 1.856, 1.917, 2.18, 2.404, 2.46...","[-0.5967, -0.5806, -0.5005, -0.3042, 0.0, 0.18...","[[0.03217, 0.03186, 0.07117, 0.0814, 0.08716, ...","[[0.08264, 0.06064, 0.0753, 0.0392, 0.03656, 0...","[[0.01704, 0.0323, 0.04984, 0.05484, 0.05585, ...","[[0.004166, 0.0266, 0.0355, 0.0466, 0.05707, 0...","[[0.001713, 0.01749, 0.01785, 0.01508, 0.01006...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.3372, 0.4973, 0.524, 0.3906, 0.4954, 0.56, ...","[0.1832, 0.4846, 0.766, 0.8545, 0.961, 0.7627,...","[0.9727, 1.524, 2.242, 2.426, 2.252, 1.847, 1....","[[0.015305, 0.0223, 0.03632, 0.02211, 0.02168,...","[[0.00202, 0.00401, 0.012985, 0.011696, 0.0151...","[[0.006813, 0.00756, 0.01591, 0.01199, 0.01014...","[[0.01706, 0.02808, 0.02686, 0.01572, 0.02292,...","[[0.01005, 0.01069, 0.01654, 0.0264, 0.02428, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [52]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

,BLA_LH_coherence_all_frequencies_all_windows,BLA_MD_coherence_all_frequencies_all_windows,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,mPFC_vHPC_coherence_all_frequencies_all_windows,...,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[[0.116, 0.34, 0.4612, 0.836, 0.853, 0.5596, 0...","[[0.4922, 0.2343, 0.4539, 0.7573, 0.831, 0.608...","[[0.627, 0.7295, 0.816, 0.951, 0.9697, 0.8765,...","[[0.625, 0.7666, 0.7227, 0.9277, 0.816, 0.6367...","[[0.8076, 0.969, 0.9854, 0.9785, 0.949, 0.91, ...","[[0.7026, 0.548, 0.8, 0.8486, 0.832, 0.581, 0....","[[0.7124, 0.767, 0.734, 0.8525, 0.99, 0.983, 0...","[[0.9395, 0.5327, 0.776, 0.81, 0.848, 0.609, 0...","[[0.959, 0.6475, 0.695, 0.807, 0.9585, 0.888, ...","[[0.9976, 0.784, 0.8037, 0.874, 0.81, 0.6714, ...",...,NaN,"[14.84, 13.97, 13.13, 12.32, 11.54, 10.78, 10....",NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[[0.116, 0.34, 0.4612, 0.836, 0.853, 0.5596, 0...","[[0.4922, 0.2343, 0.4539, 0.7573, 0.831, 0.608...","[[0.627, 0.7295, 0.816, 0.951, 0.9697, 0.8765,...","[[0.625, 0.7666, 0.7227, 0.9277, 0.816, 0.6367...","[[0.8076, 0.969, 0.9854, 0.9785, 0.949, 0.91, ...","[[0.7026, 0.548, 0.8, 0.8486, 0.832, 0.581, 0....","[[0.7124, 0.767, 0.734, 0.8525, 0.99, 0.983, 0...","[[0.9395, 0.5327, 0.776, 0.81, 0.848, 0.609, 0...","[[0.959, 0.6475, 0.695, 0.807, 0.9585, 0.888, ...","[[0.9976, 0.784, 0.8037, 0.874, 0.81, 0.6714, ...",...,"[[[40.039741738777444, 9.47029992556593], [42....","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,"[[0.727, 0.4612, 0.642, 0.9824, 0.9873, 0.984,...","[[0.8467, 0.1937, 0.177, 0.961, 0.9243, 0.8643...","[[0.5806, 0.3042, 0.5464, 0.9565, 0.872, 0.777...","[[0.773, 0.975, 0.10223, 0.147, 0.561, 0.772, ...","[[0.975, 0.7227, 0.7573, 0.9707, 0.9634, 0.931...","[[0.9614, 0.875, 0.908, 0.9316, 0.902, 0.8696,...","[[0.8076, 0.4446, 0.06335, 0.1443, 0.501, 0.67...","[[0.8833, 0.9585, 0.7524, 0.8984, 0.97, 0.9766...","[[0.81, 0.10754, 0.1426, 0.2534, 0.3474, 0.51,...","[[0.8145, 0.2239, 0.03897, 0.07855, 0.278, 0.3...",...,"[[[57.86163468579171, 31.554562356382696], [59...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,"[[0.727, 0.4612, 0.642, 0.9824, 0.9873, 0.984,...","[[0.8467, 0.1937, 0.177, 0.961, 0.9243, 0.8643...","[[0.5806, 0.3042, 0.5464, 0.9565, 0.872, 0.777...","[[0.773, 0.975, 0.10223, 0.147, 0.561, 0.772, ...","[[0.975, 0.7227, 0.7573, 0.9707, 0.9634, 0.931...","[[0.9614, 0.875, 0.908, 0.9316, 0.902, 0.8696,...","[[0.8076, 0.4446, 0.06335,

In [53]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

(39849,)

In [54]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

(18, 39849)

In [55]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

16.0

In [58]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(".", FULL_LFP_TRACES_PKL))

In [57]:
raise ValueError("")

ValueError: 

In [59]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'agent_locations', 'subject_thorax_velocity', 'agent_thorax_velocity',
       'subject_thorax_to_reward_port', 'agent_thorax_to_reward_port',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps'],
      dtype='object', length=158)